# 作業

### 嘗試用 keras 的 DepthwiseConv2D 等 layers 實做 Inverted Residual Block.
   - depthwise's filter shape 爲 (3,3), padding = same
   - 不需要給 alpha, depth multiplier 參數
   - expansion 因子爲 6

##### 載入套件

In [1]:
from tensorflow.keras.layers import Input, DepthwiseConv2D, Conv2D, BatchNormalization, ReLU, Add
from tensorflow.keras.models import Model

2024-09-12 19:54:37.475609: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


##### 定義 Separable Convolution 函數 (請在此實做)

In [4]:

def InvertedRes(input_tensor, expansion):

    # Expansion phase
    x = Conv2D(expansion, (1, 1), padding='same', use_bias=False)(input_tensor)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    # Depthwise separable convolution
    x = DepthwiseConv2D((3, 3), padding='same', use_bias=False)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    # Projection phase
    x = Conv2D(3, (1, 1), padding='same', use_bias=False)(x)
    x = BatchNormalization()(x)
    
    # Adding the input to the output
    x = Add()([input_tensor, x])
    
    return x




##### 建構模型

In [5]:
input_tensor = Input((64, 64, 3))
output_tensor = InvertedRes(input_tensor, 6)
model = Model(inputs=input_tensor, outputs=output_tensor)
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_3 (InputLayer)        [(None, 64, 64, 3)]          0         []                            
                                                                                                  
 conv2d_4 (Conv2D)           (None, 64, 64, 6)            18        ['input_3[0][0]']             
                                                                                                  
 batch_normalization_6 (Bat  (None, 64, 64, 6)            24        ['conv2d_4[0][0]']            
 chNormalization)                                                                                 
                                                                                                  
 re_lu_4 (ReLU)              (None, 64, 64, 6)            0         ['batch_normalization_6[